In [4]:
from wxpy import *

In [5]:
bot = Bot()

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as 杨少宇


In [107]:
my_self = bot.self

In [108]:
my_friend = bot.friends().search('淡渃花逝')[0]
# 缺陷1：无法针对某一个好友进行回复和识别

In [ ]:
my_friend.send('start')

In [110]:
from iexfinance import Stock
from iexfinance import get_historical_data
from datetime import datetime
import matplotlib.pyplot as plt
from iexfinance import get_available_symbols
from iexfinance import get_market_tops

In [111]:
import spacy
nlp = spacy.load('en_core_web_md')
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config
trainer = Trainer(config.load('/home/yang/文档/course/Work/nlu_config.yml'))
training_data = load_data('/home/yang/文档/course/Work/testData1.json')
interpreter = trainer.train(training_data)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished


In [121]:
@bot.register(my_friend)

def reply_my_friend(msg):
    
    state = INIT
    new_state,response = send_message(state,msg.text)
    
    
    return '{}'.format(response)

Hi
Hellow , i'm a robot who can tell you many information about stock 0
Hellow , i'm a robot who can tell you many information about stock


In [113]:
import re

def kk(a):
    if "stock_search" in a:
        a_0 = ['price', 'cap', 'volume', 'open']
        from random import choice
        return choice(a_0) 
    else :
        a_1 = re.sub(r'(_search)',"",a)
        return a_1 

In [114]:
def chengfenhuoqu(aaa):
    
    data = interpreter.parse(aaa)
    itt_1= kk(data["intent"]['name'])
    
    if itt_1 == 'cap' :
        itt_1 = 'market_cap'    
    
    params={}
    for ent in data["entities"]:
        params[ent["entity"]]=ent["value"]
        companyname = params['company']
    cpn = companyname
    
    return itt_1 , cpn

In [115]:
def huifuneirong(cpn_1,itt_2):
    
    if itt_2 == 'cap':
        itt_2 = 'market_cap'
    
    stk = Stock('{}'.format(cpn_1))
    pa_1 = "stk.get_{}()".format(itt_2)
    
    a = eval(pa_1)
    
    if itt_2 == 'market_cap' :
        itt_2 = 'cap'
    if itt_2 == 'open':
        itt_2 = 'open price'
    if itt_2 == 'volume':
        itt_2 = 'exchange volume'
    
    date_get = stk.get_quote(filter_='latestTime')['latestTime']
    
    ans = 'The {} of {} was {} in {} '.format(itt_2,cpn_1,a,date_get)
    
    return ans

In [120]:
import string

INIT = 0
GS = 1
IFMT = 2
OTHER = 3
FINAL = 4

global old_cpn

def send_message(state,message):    
#    print(message)
    #state = INIT
    if fenxijuzi(message) == "deny" or fenxijuzi(message) == "affirm":
        state = OTHER
    new_state, response = respond(state, message)
    anse=response
#    print(response,new_state)
    finalrespond = response
    if new_state == 3:
        itt,cpn = chengfenhuoqu(message)
        print(itt,cpn)
        anse = huifuneirong(cpn,itt)
        print(anse)
        finalrespond = response.format(anse)
        print(finalrespond)
#    print(finalrespond)
    return  new_state,finalrespond

def respond(state, message):
    (new_state, response) = policy[(state, fenxijuzi(message))]
    return new_state, response

policy = {
    (INIT, "greet"):(INIT, "Hellow , i'm a robot who can tell you many information about stock"),
    (INIT, "getsymbol"):(OTHER, "Ok , i can provide you some information of that stock, just like...{},do you need other information ?"),
    (INIT, "getkind"):(OTHER, "{} , do you need other information ?"),
    (OTHER, "affirm"):(INIT, "well , tell me more"),
    (OTHER, "deny"):(FINAL, "OK , good luck , bye"),
}


In [119]:
def fenxijuzi(sss):
    data = interpreter.parse(sss)
    itt_1 = data["intent"]['name']
    msg = sss.lower()
    
    
    if msg == "no" or msg == "no thanks":
        return 'deny'
    if msg == "yes" or msg == "sure" or msg == "ok":
        return 'affirm'
    if itt_1 == "greet":
        return 'greet'
    if itt_1 == "stock_search":
        return 'getsymbol'
    if itt_1 == "cap_search" or itt_1 == "volume_search" or itt_1 == "price_search":
        return 'getkind'
    